In [1]:
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
import keras
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
(keras.__version__, tf.__version__)

Using TensorFlow backend.


('2.2.2', '1.10.1')

In [3]:
exch = 'BTRX'
pair = 'BTC/USDT'

df = pd.read_csv(f"{exch}_{pair.replace('/', '-')}_ohlcv.csv")
df.head()

,timestamp,open,high,low,close,volume
0,2016-12-31 05:02:00,948.000,948.000,942.899,942.899,0.083403
1,2016-12-31 05:03:00,942.899,942.899,942.899,942.899,0.000000
2,2016-12-31 05:04:00,942.899,942.899,942.899,942.899,0.000000
3,2016-12-31 05:05:00,942.899,942.899,942.899,942.899,0.000000
4,2016-12-31 05:06:00,942.899,942.899,942.899,942.899,0.000000


In [4]:
df.dtypes

timestamp     object
open         float64
high         float64
low          float64
close        float64
volume       float64
dtype: object

In [5]:
# Plot the stock prices for the last day
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates

plot_last_n_minutes = 60 * 12  # 1/2 day
cs_frame = df.iloc[-1 * plot_last_n_minutes:].copy()  # Create the candlestick frame

#if necessary convert to datetime
cs_frame.timestamp = pd.to_datetime(cs_frame.timestamp)

cs_frame = cs_frame[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
cs_frame["timestamp"] = cs_frame["timestamp"].apply(mdates.date2num)

f1 = plt.subplot2grid((6, 1), (0, 0), rowspan=6, colspan=1, facecolor='#07000d')
candlestick_ohlc(f1, cs_frame.values, width=.0001, colorup='#53c156', colordown='#ff1717', alpha=.75)
f1.xaxis_date()
f1.xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d %H:%M'))

plt.xticks(rotation=45)
plt.ylabel('Stock Price')
plt.xlabel('Date Hours:Minutes')
plt.show()

# Cleanup memory
%reset_selective -f "^cs_frame$"

<Figure size 640x480 with 1 Axes>

In [6]:
# Add a minute moving average over period
def add_moving_avg(df, period=30):
    #df[f"{period}_ma"] = pd.rolling_mean(df['close'], period)
    df[f"{period}_ma"] = df.close.rolling(period).mean()

In [7]:
add_moving_avg(df)
df.tail()

,timestamp,open,high,low,close,volume,30_ma
532493,2018-01-04 23:55:00,15199.000000,15200.00000,15199.000000,15199.000000,10.446506,15063.411186
532494,2018-01-04 23:56:00,15199.000000,15220.00000,15195.582639,15220.000000,5.457758,15070.777853
532495,2018-01-04 23:57:00,15220.000000,15238.00000,15200.000000,15201.000000,7.473745,15077.644520
532496,2018-01-04 23:58:00,15200.000000,15202.21529,15085.001000,15101.591266,7.258691,15081.030929
532497,2018-01-04 23:59:00,15101.591266,15199.00000,15085.001000,15199.000000,1.777752,15087.697595


In [8]:
from stock_gym.envs import stocks
import gym

env = gym.make('ContSinMarketEnv-v0')
#env.add_data(df)

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, GRU, Dropout
from keras.optimizers import Adam

In [10]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [11]:
np.random.seed(123)
env.seed(123)
(env.n_actions, env.observation_shape())

(3, (1, 64))

In [12]:
model = Sequential()

model.add(GRU(16,
                #input_shape=env.observation_space.shape,
                input_shape=env.observation_shape(),
                dropout=0.1,
                recurrent_dropout=0.5,
                #return_sequences=True,
               #stateful=False,
             ))
# model.add(GRU(128,
#                 dropout=0.1,
#                 recurrent_dropout=0.5,
#                 return_sequences=True,
#              ))
# model.add(GRU(128,
#                 dropout=0.1,
#                 recurrent_dropout=0.5,
#              ))
#model.add(Dense(64))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))

model.add(Dense(env.n_actions, kernel_initializer='lecun_uniform', activation='linear'))
#model.add(Activation('linear')) #linear output so we can have range of real-valued outputs
#model.add(Dense(env.n_actions, activation='linear'))

# model.add(Flatten(input_shape=env.observation_space.shape))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(16))
# model.add(Activation('relu'))
# model.add(Dense(env.n_actions, activation='linear'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 16)                3888      
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 3,939
Trainable params: 3,939
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
memory = SequentialMemory(limit=env.total_space_size, window_length=env.n_features)
policy = BoltzmannQPolicy()

In [14]:
dqn = DQNAgent(model=model, nb_actions=env.n_actions, memory=memory, nb_steps_warmup=100,
               enable_dueling_network=True, dueling_type='avg', target_model_update=1e-2, policy=policy)


In [15]:
#dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.compile(
    #loss='mse',
    optimizer=Adam(lr=1e-3),
    metrics=['mae'],
)

In [16]:
history = dqn.fit(env, nb_steps=5000, visualize=False, verbose=2)

Training for 5000 steps ...
    2/5000: episode: 1, duration: 0.221s, episode steps: 2, steps per second: 9, episode reward: -1000.003, mean reward: -500.001 [-1000.002, -0.001], mean action: 1.000 [0.000, 2.000], mean observation: 0.728 [0.151, 1.000], loss: --, mean_absolute_error: --, mean_q: --
    3/5000: episode: 2, duration: 0.004s, episode steps: 1, steps per second: 251, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.787 [0.296, 1.000], loss: --, mean_absolute_error: --, mean_q: --
    4/5000: episode: 3, duration: 0.002s, episode steps: 1, steps per second: 432, episode reward: -1000.002, mean reward: -1000.002 [-1000.002, -1000.002], mean action: 2.000 [2.000, 2.000], mean observation: 0.392 [0.000, 0.966], loss: --, mean_absolute_error: --, mean_q: --
    5/5000: episode: 4, duration: 0.003s, episode steps: 1, steps per second: 362, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action

   82/5000: episode: 81, duration: 0.003s, episode steps: 1, steps per second: 366, episode reward: -1000.000, mean reward: -1000.000 [-1000.000, -1000.000], mean action: 1.000 [1.000, 1.000], mean observation: 0.290 [0.000, 0.867], loss: --, mean_absolute_error: --, mean_q: --
   83/5000: episode: 82, duration: 0.003s, episode steps: 1, steps per second: 358, episode reward: -1000.001, mean reward: -1000.001 [-1000.001, -1000.001], mean action: 2.000 [2.000, 2.000], mean observation: 0.808 [0.388, 1.000], loss: --, mean_absolute_error: --, mean_q: --
   84/5000: episode: 83, duration: 0.002s, episode steps: 1, steps per second: 421, episode reward: -1000.002, mean reward: -1000.002 [-1000.002, -1000.002], mean action: 2.000 [2.000, 2.000], mean observation: 0.529 [0.003, 1.000], loss: --, mean_absolute_error: --, mean_q: --
   85/5000: episode: 84, duration: 0.003s, episode steps: 1, steps per second: 388, episode reward: -1000.000, mean reward: -1000.000 [-1000.000, -1000.000], mean 

  115/5000: episode: 114, duration: 0.015s, episode steps: 1, steps per second: 67, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.622 [0.043, 1.000], loss: 280360.968750, mean_absolute_error: 188.293259, mean_q: -0.859543
  116/5000: episode: 115, duration: 0.016s, episode steps: 1, steps per second: 61, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.627 [0.047, 1.000], loss: 358000.375000, mean_absolute_error: 240.242218, mean_q: -0.984811
  117/5000: episode: 116, duration: 0.014s, episode steps: 1, steps per second: 69, episode reward: -1000.000, mean reward: -1000.000 [-1000.000, -1000.000], mean action: 1.000 [1.000, 1.000], mean observation: 0.280 [0.000, 0.852], loss: 357930.843750, mean_absolute_error: 240.283051, mean_q: -1.010729
  118/5000: episode: 117, duration: 0.014s, episode steps: 1, steps per second: 72, episode reward: -0.001, m

  143/5000: episode: 142, duration: 0.014s, episode steps: 1, steps per second: 71, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.775 [0.259, 1.000], loss: 324030.781250, mean_absolute_error: 220.573090, mean_q: -1.670188
  144/5000: episode: 143, duration: 0.016s, episode steps: 1, steps per second: 61, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.748 [0.196, 1.000], loss: 339729.562500, mean_absolute_error: 230.808472, mean_q: -1.528084
  145/5000: episode: 144, duration: 0.014s, episode steps: 1, steps per second: 73, episode reward: -1000.001, mean reward: -1000.001 [-1000.001, -1000.001], mean action: 2.000 [2.000, 2.000], mean observation: 0.184 [0.000, 0.552], loss: 262498.625000, mean_absolute_error: 179.601578, mean_q: -1.596015
  146/5000: episode: 145, duration: 0.014s, episode steps: 1, steps per second: 70, episode reward: -0.001, m

  171/5000: episode: 170, duration: 0.016s, episode steps: 1, steps per second: 62, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.640 [0.056, 1.000], loss: 291887.062500, mean_absolute_error: 200.623703, mean_q: -1.289813
  172/5000: episode: 171, duration: 0.015s, episode steps: 1, steps per second: 67, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.535 [0.005, 1.000], loss: 230656.171875, mean_absolute_error: 159.553955, mean_q: -1.211856
  173/5000: episode: 172, duration: 0.014s, episode steps: 1, steps per second: 70, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.609 [0.034, 1.000], loss: 307063.375000, mean_absolute_error: 210.810852, mean_q: -1.263484
  174/5000: episode: 173, duration: 0.014s, episode steps: 1, steps per second: 71, episode reward: -0.001, mean reward: 

...

 4823/5000: episode: 4822, duration: 0.014s, episode steps: 1, steps per second: 74, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.218 [0.000, 0.722], loss: 0.000002, mean_absolute_error: 41.741936, mean_q: -0.000883
 4824/5000: episode: 4823, duration: 0.015s, episode steps: 1, steps per second: 67, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.459 [0.000, 0.994], loss: 0.000005, mean_absolute_error: 41.736801, mean_q: -0.000872
 4825/5000: episode: 4824, duration: 0.015s, episode steps: 1, steps per second: 69, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.791 [0.308, 1.000], loss: 0.000002, mean_absolute_error: 41.735985, mean_q: -0.000851
 4826/5000: episode: 4825, duration: 0.014s, episode steps: 1, steps per second: 70, episode reward: -0.001, mean reward: -0.001 [-0.001

 4861/5000: episode: 4860, duration: 0.020s, episode steps: 1, steps per second: 50, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.804 [0.362, 1.000], loss: 0.000001, mean_absolute_error: 41.740776, mean_q: -0.000894
 4862/5000: episode: 4861, duration: 0.018s, episode steps: 1, steps per second: 55, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.185 [0.000, 0.555], loss: 0.000005, mean_absolute_error: 41.733192, mean_q: -0.000843
 4863/5000: episode: 4862, duration: 0.017s, episode steps: 1, steps per second: 59, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.187 [0.000, 0.581], loss: 0.000005, mean_absolute_error: 41.735046, mean_q: -0.000819
 4864/5000: episode: 4863, duration: 0.017s, episode steps: 1, steps per second: 57, episode reward: -0.001, mean reward: -0.001 [-0.001

 4899/5000: episode: 4898, duration: 0.015s, episode steps: 1, steps per second: 66, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.501 [0.000, 1.000], loss: 0.000002, mean_absolute_error: 41.738293, mean_q: -0.000864
 4900/5000: episode: 4899, duration: 0.017s, episode steps: 1, steps per second: 60, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.742 [0.184, 1.000], loss: 0.000003, mean_absolute_error: 41.737694, mean_q: -0.000852
 4901/5000: episode: 4900, duration: 0.015s, episode steps: 1, steps per second: 65, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.702 [0.121, 1.000], loss: 0.000003, mean_absolute_error: 41.738205, mean_q: -0.000848
 4902/5000: episode: 4901, duration: 0.014s, episode steps: 1, steps per second: 73, episode reward: -0.001, mean reward: -0.001 [-0.001

 4940/5000: episode: 4939, duration: 0.019s, episode steps: 1, steps per second: 52, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.238 [0.000, 0.775], loss: 0.000002, mean_absolute_error: 41.742748, mean_q: -0.000954
 4941/5000: episode: 4940, duration: 0.020s, episode steps: 1, steps per second: 51, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.352 [0.000, 0.937], loss: 0.000002, mean_absolute_error: 41.742893, mean_q: -0.000905
 4942/5000: episode: 4941, duration: 0.016s, episode steps: 1, steps per second: 63, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.311 [0.000, 0.894], loss: 0.000001, mean_absolute_error: 41.745308, mean_q: -0.000948
 4943/5000: episode: 4942, duration: 0.017s, episode steps: 1, steps per second: 59, episode reward: -0.001, mean reward: -0.001 [-0.001

 4976/5000: episode: 4975, duration: 0.019s, episode steps: 1, steps per second: 52, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.484 [0.000, 0.999], loss: 0.000003, mean_absolute_error: 41.735519, mean_q: -0.000849
 4977/5000: episode: 4976, duration: 0.019s, episode steps: 1, steps per second: 52, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.586 [0.022, 1.000], loss: 0.000008, mean_absolute_error: 41.734226, mean_q: -0.000958
 4978/5000: episode: 4977, duration: 0.016s, episode steps: 1, steps per second: 61, episode reward: -0.001, mean reward: -0.001 [-0.001, -0.001], mean action: 0.000 [0.000, 0.000], mean observation: 0.384 [0.000, 0.961], loss: 0.000007, mean_absolute_error: 41.729748, mean_q: -0.000873
 4979/5000: episode: 4978, duration: 0.015s, episode steps: 1, steps per second: 67, episode reward: -0.001, mean reward: -0.001 [-0.001

In [17]:
# After training is done, we save the final weights.
dqn.save_weights('duel_dqn_{}_weights.h5f'.format('stock_bot'), overwrite=True)

In [18]:
dqn.test(env, nb_episodes=5, visualize=False)

Testing for 5 episodes ...
Episode 1: reward: -0.001, steps: 1
Episode 2: reward: -0.001, steps: 1
Episode 3: reward: -0.001, steps: 1
Episode 4: reward: -0.001, steps: 1
Episode 5: reward: -0.001, steps: 1
